# APM598: Homework 3 (03/26)
## author: Jieshu Wang (jwang490@asu.edu)

# 1. n-gram models
## Ex 1.
### Ex 1. a) 
- Load and tokenize the text attached ’Plato_Republic.txt’.
- Put all the words in lower case to regroup words like ’The’ and ’the’.
- Compute the total number of words N in the text and the number of unique words (size of the vocabulary).
    - size of vocabulary is ___7544___.

In [1]:
with open('./data/Plato_Republic.txt', mode='r', encoding='utf-8-sig') as my_file:
    # encoding='utf-8-sig' is to remove '\ufeff' in the text.
    my_text = my_file.read().replace('\n',' ')
    my_text = my_text.lower()

print(my_text[0:44])

the republic.     persons of the dialogue.  


In [2]:
# Tokenize words
import nltk
nltk.data.path.append("/Users/Jieshu/PycharmProjects/APM598_Deep_Neural_Networks/APM598_notebook4/nltk_data")

my_text_tokenized = nltk.word_tokenize(my_text)
print(my_text_tokenized[0:10])
print("-- length of texts (words) : ",len(my_text_tokenized))

['the', 'republic', '.', 'persons', 'of', 'the', 'dialogue', '.', 'socrates', ',']
-- length of texts (words) :  136312


### Ex 1. b) 
- Build a uni-gram. 
- Deduce the 5 most common words with at least 8 characters. 
    - 'certainly', 'knowledge', 'injustice', 'therefore', 'question'

In [3]:
# build a uni-gram
my_unigram = nltk.ngrams(my_text_tokenized, 1)
freq_dist_uni = nltk.FreqDist(my_unigram)

# deduce the 5 most common words with at least 8 characters
sorted_freq_dist_uni = freq_dist_uni.most_common()
sorted_8_character_words = [(i[0][0], i[1]) for i in sorted_freq_dist_uni if len(i[0][0]) >=8]
print("The 5 most frequent words with at least 8 characters are:")
print(sorted_8_character_words[0:5])
print(f'The number of unique words are {freq_dist_uni.B()}')

The 5 most frequent words with at least 8 characters are:
[('certainly', 235), ('knowledge', 152), ('injustice', 111), ('therefore', 109), ('question', 99)]
The number of unique words are 7544


### Ex 1. c)
- Build a bi-gram and define a function that given two words (x1, x2) compute the probability
: $\mathbb{P}(x_{t+1}|x_t) = \frac{\#\{(x_t,x_{t+1})\}}{\#\{(x_t)\}\}}$.

In [4]:
my_bigram = nltk.ngrams(my_text_tokenized, 2)
# define a function to compute the probability
def prob_word_pair(x_1: str, x_2: str, text) ->float:
    corpus = nltk.word_tokenize(text.lower())
    unigram = nltk.ngrams(corpus, 1)
    bigram = nltk.ngrams(corpus, 2)
    freq_distribution_uni = nltk.FreqDist(unigram)
    freq_distribution_bi = nltk.FreqDist(bigram)
    n_x_1 = freq_distribution_uni[(x_1.lower(),)]
    n_x_1_2 = freq_distribution_bi[(x_1.lower(), x_2.lower())]
    prob = n_x_1_2/n_x_1
    return prob

print(prob_word_pair('I', 'think', my_text))

0.032217308907138344


## Ex 1. d) 
- Deduce the so-called perplexity of the bi-gram model
    - 37.89928782828084
    

In [5]:
def compute_perplexity(text)->float:
    corpus = nltk.word_tokenize(text.lower())
    corpus_length = len(corpus)
    unigram = nltk.ngrams(corpus, 1)
    bigram = nltk.ngrams(corpus, 2)
    freq_distribution_uni = nltk.FreqDist(unigram)
    freq_distribution_bi = nltk.FreqDist(bigram)
    product = 1
    for i in range(corpus_length-1):
        x_1 = corpus[i]
        x_2 = corpus[i+1]
        n_x_1 = freq_distribution_uni[(x_1.lower(),)]
        n_x_1_2 = freq_distribution_bi[(x_1.lower(), x_2.lower())]
        prob = n_x_1_2/n_x_1
        product *= prob**(-1/(corpus_length-1))
    perplexity = product
    return  perplexity

print("The perplexity of Plato's Republic is: ")
print(compute_perplexity(my_text))

The perplexity of Plato's Republic is: 
37.89928782828084


# 2. Recurrent Neural Networks
## Ex 2. 
### Ex 2. a) find the output y1, . . . , y5 and deduce the predicted characters

In [1]:
from HW3_util import ModuleParamsVanillaRNN, VanillaRNN, letter_to_tensor, torch

import torch.nn.functional as F

a_ex2a = torch.tensor([[1.0, -1.0, -0.5, 0.5],
                       [1.0, 1.0, -0.5, -1]])
r_ex2a = torch.tensor([[1.0, 0.0],
                      [0.0, 1.0]])
b_ex2a = torch.tensor([[1.0, 1.0],
                       [0.5, 1.0],
                       [-1.0, 0.0],
                       [0.0, -0.5]])
all_letters_ex2a = 'helo'
input_letter_list = ['h', 'e', 'l', 'l', 'o']

class VanillaRNNEx2a(VanillaRNN):
    def __init__(self, module_params: ModuleParamsVanillaRNN):
        super(VanillaRNNEx2a, self).__init__(module_params)
        with torch.no_grad():
            self._A.weight.data = a_ex2a
            # self._A.bias.data.fill_(0)
            self._R.weight.data = r_ex2a
            # self._R.bias.data.fill_(0)
            self._B.weight.data = b_ex2a
            # self._B.bias.data.fill_(0)

def predict_characters_ex2a(input_list: list, all_letters: str):
    vanilla_rnn_params = ModuleParamsVanillaRNN(all_letters_base=all_letters)
    my_vanilla_rnn_ex2a = VanillaRNNEx2a(vanilla_rnn_params)
    h = my_vanilla_rnn_ex2a.init_hidden()
    predicted_charaters_list = []
    for i in range(len(input_list)):
        input = input_list[i]
        print(f'--- {i}---')
        x = letter_to_tensor(input, all_letters)
        y, h = my_vanilla_rnn_ex2a(x, h)
        predicted_character = all_letters[y.argmax()]
        print(f"    input: {input}")
        print(f'    y_{i+1}: {y}')
        print(f'    output: {predicted_character}')
        predicted_charaters_list.append(predicted_character)
    print(f'Predicted characters are: {predicted_charaters_list}')

print('the output y1, . . . , y5 and deduce the predicted characters')
predict_characters_ex2a(input_letter_list, all_letters_ex2a)

the output y1, . . . , y5 and deduce the predicted characters
--- 0---
    input: h
    y_1: tensor([[0.5170, 0.3533, 0.0526, 0.0770]], grad_fn=<SoftmaxBackward>)
    output: h
--- 1---
    input: e
    y_2: tensor([[0.3275, 0.3681, 0.2037, 0.1006]], grad_fn=<SoftmaxBackward>)
    output: e
--- 2---
    input: l
    y_3: tensor([[0.1762, 0.2410, 0.4063, 0.1765]], grad_fn=<SoftmaxBackward>)
    output: l
--- 3---
    input: l
    y_4: tensor([[0.0949, 0.1423, 0.5210, 0.2418]], grad_fn=<SoftmaxBackward>)
    output: l
--- 4---
    input: o
    y_5: tensor([[0.0940, 0.1092, 0.3790, 0.4178]], grad_fn=<SoftmaxBackward>)
    output: o
Predicted characters are: ['h', 'e', 'l', 'l', 'o']


### Ex 2. b) Find matrices A, R, B such that the predicted characters are ”olleh”.
- A.weight: [[-1.6251, -0.6183,  0.7184,  1.0523], [-1.5250, -0.3831,  1.4286, -0.9764]]
- R.weight: [[ 2.6304,  1.8039], [-1.3376, -0.0809]]
- B.weight: [[ 2.6276, -1.8918], [ 2.9543,  2.2045], [-2.1664,  2.6917], [-1.7653, -2.3543]]

In [1]:
from HW3_util import VanillaRNNTrainer, VanillaRNN, ModuleParamsVanillaRNN

all_letters_ex2b = 'helo'
training_x_ex2b = 'hello'
training_y_ex2b = 'olleh'
module_params_ex2b = ModuleParamsVanillaRNN(all_letters_base=all_letters_ex2b, chunk_size=5,
                                            is_print_training=True, n_steps=600, printing_step=60)
trainer_ex2b = VanillaRNNTrainer(module_params=module_params_ex2b, module=VanillaRNN,
                                 training_x=training_x_ex2b, training_y=training_y_ex2b)
print(trainer_ex2b.model.parameters())

  input = hello
  output = ooooo
loss at step 0: 1.391243076324463
  input = hello
  output = ooeeh
loss at step 60: 1.3292919158935548
  input = hello
  output = olheh
loss at step 120: 1.1782100677490235
  input = hello
  output = olleh
loss at step 180: 1.0148378372192384
  input = hello
  output = olleh
loss at step 240: 0.873841381072998
  input = hello
  output = olleh
loss at step 300: 0.827212905883789
  input = hello
  output = olleh
loss at step 360: 0.8035863876342774
  input = hello
  output = olleh
loss at step 420: 0.7893017292022705
  input = hello
  output = olleh
loss at step 480: 0.779842472076416
  input = hello
  output = olleh
loss at step 540: 0.7731913566589356
<generator object Module.parameters at 0x132e26750>


In [10]:
result_model_parameters = trainer_ex2b.model.named_parameters()
for name, params in result_model_parameters:
    print(f'{name}')
    print(f'{params}')

_A.weight
    Parameter containing:
tensor([[-1.6251, -0.6183,  0.7184,  1.0523],
        [-1.5250, -0.3831,  1.4286, -0.9764]], requires_grad=True)
_R.weight
    Parameter containing:
tensor([[ 2.6304,  1.8039],
        [-1.3376, -0.0809]], requires_grad=True)
_B.weight
    Parameter containing:
tensor([[ 2.6276, -1.8918],
        [ 2.9543,  2.2045],
        [-2.1664,  2.6917],
        [-1.7653, -2.3543]], requires_grad=True)


## Ex 3. vanishing/exploding gradient




